# Data Wrangling for Capstone 3 - Diamond Price Data

In [1]:
# Import Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

### 1.0 Data Collection

In [2]:
# File path locations & Read into Dataframe(s)

diamonds_file = '../Data/diamonds.csv'

diamonds_df = pd.read_csv(diamonds_file)

In [3]:
# View Loaded Dataframe

diamonds_df.sample(10), diamonds_df.shape

carat        cut color clarity  depth  table     x     y     z  price
24151   1.57      Ideal     F     SI1   61.1   55.0  7.53  7.55  4.61  12320
3486    0.90    Premium     I     VS2   61.8   59.0  6.15  6.21  3.82   3398
51768   0.70      Ideal     F     SI1   62.1   55.0  5.68  5.72  3.54   2409
27064   2.03      Ideal     G     SI2   61.1   57.0  8.16  8.21  5.00  17182
52554   0.70      Ideal     F     VS2   62.0   57.0  5.71  5.67  3.53   2535
27826   0.39  Very Good     I     VS2   62.9   57.0  4.64  4.68  2.93    650
255     1.05       Fair     J     SI2   65.8   59.0  6.41  6.27  4.18   2789
37467   0.42      Ideal     G     VS1   62.7   54.0  4.83  4.80  3.02    984
17920   1.31       Good     I     VS2   62.5   56.0  7.03  6.98  4.38   7236
20020   0.30       Good     E     SI2   63.5   55.0  4.27  4.30  2.72    421

### 2.0 Data Definition

In [4]:
diamonds_df.dtypes, diamonds_df.shape

(carat      float64
 cut         object
 color       object
 clarity     object
 depth      float64
 table      float64
 x          float64
 y          float64
 z          float64
 price        int64
 dtype: object,
 (53940, 10))

In [5]:
# Check missing/null values

missing_diamond = pd.concat([diamonds_df.isnull().sum(), 100 * diamonds_df.isnull().mean()], axis=1)
missing_diamond.columns=['count', '%']
missing_diamond.sort_values(by='count', ascending=False)

count    %
carat        0  0.0
cut          0  0.0
color        0  0.0
clarity      0  0.0
depth        0  0.0
table        0  0.0
x            0  0.0
y            0  0.0
z            0  0.0
price        0  0.0

In [7]:
nonzero_cols = diamonds_df.astype(bool).sum(axis=0)

nonzero_cols

carat      53940
cut        53940
color      53940
clarity    53940
depth      53940
table      53940
x          53932
y          53933
z          53920
price      53940
dtype: int64

#### *Good news is that there are no null values to deal with here. There are some records in the 'x', 'y', and 'z' columns that have values of zero and likely to cause an issue in EDA and modeling. Only a small number of rows affected, going to drop these.*

### 3.0 Data Cleaning

In [8]:
# Identify rows where 'x' values are zero

zeros_x = diamonds_df[diamonds_df['x'] == 0]

zeros_x

carat        cut color clarity  depth  table    x     y    z  price
11182   1.07      Ideal     F     SI2   61.6   56.0  0.0  6.62  0.0   4954
11963   1.00  Very Good     H     VS2   63.3   53.0  0.0  0.00  0.0   5139
15951   1.14       Fair     G     VS1   57.5   67.0  0.0  0.00  0.0   6381
24520   1.56      Ideal     G     VS2   62.2   54.0  0.0  0.00  0.0  12800
26243   1.20    Premium     D    VVS1   62.1   59.0  0.0  0.00  0.0  15686
27429   2.25    Premium     H     SI2   62.8   59.0  0.0  0.00  0.0  18034
49556   0.71       Good     F     SI2   64.1   60.0  0.0  0.00  0.0   2130
49557   0.71       Good     F     SI2   64.1   60.0  0.0  0.00  0.0   2130

In [9]:
# Identify rows where 'z' values are zero

zeros_z = diamonds_df[diamonds_df['z'] == 0]

zeros_z

carat        cut color clarity  depth  table     x     y    z  price
2207    1.00    Premium     G     SI2   59.1   59.0  6.55  6.48  0.0   3142
2314    1.01    Premium     H      I1   58.1   59.0  6.66  6.60  0.0   3167
4791    1.10    Premium     G     SI2   63.0   59.0  6.50  6.47  0.0   3696
5471    1.01    Premium     F     SI2   59.2   58.0  6.50  6.47  0.0   3837
10167   1.50       Good     G      I1   64.0   61.0  7.15  7.04  0.0   4731
11182   1.07      Ideal     F     SI2   61.6   56.0  0.00  6.62  0.0   4954
11963   1.00  Very Good     H     VS2   63.3   53.0  0.00  0.00  0.0   5139
13601   1.15      Ideal     G     VS2   59.2   56.0  6.88  6.83  0.0   5564
15951   1.14       Fair     G     VS1   57.5   67.0  0.00  0.00  0.0   6381
24394   2.18    Premium     H     SI2   59.4   61.0  8.49  8.45  0.0  12631
24520   1.56      Ideal     G     VS2   62.2   54.0  0.00  0.00  0.0  12800
26123   2.25    Premium     I     SI1   61.3   58.0  8.52  8.42  0.0  15397
26243   1.20    Premium     D    VVS1   62.1   59.0  0.00  0.00  0.0  15686
27112   2.20    Premium     H     SI1   61.2   59.0  8.42  8.37  0.0  17265
27429   2.25    Premium     H     SI2   62.8   59.0  0.00  0.00  0.0  18034
27503   2.02    Premium     H     VS2   62.7   53.0  8.02  7.95  0.0  18207
27739   2.80       Good     G     SI2   63.8   58.0  8.90  8.85  0.0  18788
49556   0.71       Good     F     SI2   64.1   60.0  0.00  0.00  0.0   2130
49557   0.71       Good     F     SI2   64.1   60.0  0.00  0.00  0.0   2130
51506   1.12    Premium     G      I1   60.4   59.0  6.71  6.67  0.0   2383

In [10]:
zeros_z.shape

(20, 10)

In [21]:
# Drop rows where 'z' values are zero

diamonds_drop = diamonds_df.drop(diamonds_df[diamonds_df['z'] == 0].index)
diamonds_drop.reset_index(drop=True, inplace=True)


diamonds_drop

carat        cut color clarity  depth  table     x     y     z  price
0       0.23      Ideal     E     SI2   61.5   55.0  3.95  3.98  2.43    326
1       0.21    Premium     E     SI1   59.8   61.0  3.89  3.84  2.31    326
2       0.23       Good     E     VS1   56.9   65.0  4.05  4.07  2.31    327
3       0.29    Premium     I     VS2   62.4   58.0  4.20  4.23  2.63    334
4       0.31       Good     J     SI2   63.3   58.0  4.34  4.35  2.75    335
...      ...        ...   ...     ...    ...    ...   ...   ...   ...    ...
53915   0.72      Ideal     D     SI1   60.8   57.0  5.75  5.76  3.50   2757
53916   0.72       Good     D     SI1   63.1   55.0  5.69  5.75  3.61   2757
53917   0.70  Very Good     D     SI1   62.8   60.0  5.66  5.68  3.56   2757
53918   0.86    Premium     H     SI2   61.0   58.0  6.15  6.12  3.74   2757
53919   0.75      Ideal     D     SI2   62.2   55.0  5.83  5.87  3.64   2757

[53920 rows x 10 columns]

#### *Dropped rows where column 'z' values are zero, which also takes care of the 'x' and 'y' columns with zero values.*

In [22]:
diamonds_drop.describe()

carat         depth         table             x             y  \
count  53920.000000  53920.000000  53920.000000  53920.000000  53920.000000   
mean       0.797698     61.749514     57.456834      5.731627      5.734887   
std        0.473795      1.432331      2.234064      1.119423      1.140126   
min        0.200000     43.000000     43.000000      3.730000      3.680000   
25%        0.400000     61.000000     56.000000      4.710000      4.720000   
50%        0.700000     61.800000     57.000000      5.700000      5.710000   
75%        1.040000     62.500000     59.000000      6.540000      6.540000   
max        5.010000     79.000000     95.000000     10.740000     58.900000   

                  z         price  
count  53920.000000  53920.000000  
mean       3.540046   3930.993231  
std        0.702530   3987.280446  
min        1.070000    326.000000  
25%        2.910000    949.000000  
50%        3.530000   2401.000000  
75%        4.040000   5323.250000  
max       31.800000  18823.000000

In [24]:
diamonds_drop.cut.unique()

array(['Ideal', 'Premium', 'Good', 'Very Good', 'Fair'], dtype=object)

In [25]:
diamonds_drop.color.unique()

array(['E', 'I', 'J', 'H', 'F', 'G', 'D'], dtype=object)

In [26]:
diamonds_drop.clarity.unique()

array(['SI2', 'SI1', 'VS1', 'VS2', 'VVS2', 'VVS1', 'I1', 'IF'],
      dtype=object)

#### *No issues with the categorical columns as far as I can tell; no blank spaces, duplicates, or mispellings to deal with.*

### 4.0 Exporting Clean Dataframes to CSV

In [31]:
# Export Clean Files

diamonds_drop.to_csv('../Data/diamonds_clean.csv', index=False)